<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/adv_cv/class_2/11%20-%20%20Laborat%C3%B3rio/C3_W2_Lab_2_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Visão computacional avançada com TensorFlow](https://www.coursera.org/learn/advanced-computer-vision-with-tensorflow?specialization=tensorflow-advanced-techniques) de [Laurence Moroney](https://laurencemoroney.com/) e [Andrew Ng](https://www.coursera.org/instructor/andrewng) , [DeepLearning.AI](https://www.deeplearning.ai/)

# Detecção de Objetos


Este laboratório é semelhante ao anterior, só que agora, em vez de imprimir as coordenadas da caixa delimitadora, você também pode visualizar essas caixas delimitadoras sobre a imagem!

## Configuração

In [ ]:
# Para executar a inferência no módulo TF-Hub.
import tensorflow as tf

import tensorflow_hub as hub

# Para fazer o download da imagem.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# Para desenhar na imagem.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# Para medir o tempo de inferência.
import time

# Imprimir a versão do Tensorflow
print(tf.__version__)

# Verificar os dispositivos de GPU disponíveis.
print("Os seguintes dispositivos de GPU estão disponíveis: %s" % tf.test.gpu_device_name())

### Selecionar e carregar o modelo
Como no laboratório anterior, você pode escolher um módulo de detecção de objetos. Aqui estão dois que selecionados para você:
* [ssd + mobilenet V2](https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2) pequeno e rápido.
* [FasterRCNN + InceptionResNet V2](https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1): alta precisão

In [ ]:
# você pode trocar as linhas comentadas aqui para escolher o outro modelo

# ssd mobilenet versão 2
module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"

# Em vez disso, você pode escolher a versão 2 do resnet inicial
#module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

#### Carregar o modelo

Em seguida, você carregará o modelo especificado pelo `module_handle`.
- Isso levará alguns minutos para carregar o modelo.

In [ ]:
model = hub.load(module_handle)

#### Escolha a assinatura padrão

Como antes, você pode verificar as assinaturas disponíveis usando `.signature.keys()`

In [ ]:
# Dê uma olhada nas assinaturas disponíveis para esse mod específico
model.signatures.keys()

Escolha a assinatura "padrão" para seu detector de objetos.

In [ ]:
detector = model.signatures['default']

### download_and_resize_image

Como você viu no laboratório anterior, essa função faz o download de uma imagem especificada por uma determinada "url", pré-processa-a e salva-a no disco.
- A novidade em relação ao laboratório anterior é que você pode exibir a imagem se definir o parâmetro `display=True`.

In [ ]:
def display_image(image):
    """
    Exibe uma imagem dentro do notebook.
    Isso é usado por download_and_resize_image()
    """
    fig = plt.figure(figsize=(20, 15))
    plt.grid(False)
    plt.imshow(image)


def download_and_resize_image(url, new_width=256, new_height=256, display=False):
    '''
    Obtém uma imagem on-line, redimensiona-a e salva-a localmente.
    
    Args:
        url (string) -- link para a imagem
        new_width (int) -- tamanho em pixels usado para redimensionar a largura da imagem
        new_height (int) -- tamanho em pixels usado para redimensionar o comprimento da imagem
        
    Retorna:
        (string) -- caminho para a imagem salva
    '''
    
    
    # cria um arquivo temporário que termina com ".jpg"
    _, filename = tempfile.mkstemp(suffix=".jpg")
    
    # abre o URL fornecido
    response = urlopen(url)
    
    # lê a imagem obtida do URL
    image_data = response.read()
    
    # Coloca os dados da imagem no buffer de memória
    image_data = BytesIO(image_data)
    
    # abre a imagem
    pil_image = Image.open(image_data)
    
    # Redimensiona a imagem. Será cortada se a proporção for diferente.
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.Resampling.LANCZOS)
    
    # converte para o espaço de cores RGB
    pil_image_rgb = pil_image.convert("RGB")
    
    # salva a imagem no arquivo temporário criado anteriormente
    pil_image_rgb.save(filename, format="JPEG", quality=90)
    
    print("Imagem baixada para %s." % filename)
    
    if display:
        display_image(pil_image)

    
    return filename

### Selecionar e carregar uma imagem
Carregue uma imagem pública do Open Images v4, salve localmente e exiba.

In [ ]:
# Por Heiko Gorski, Fonte: https://commons.wikimedia.org/wiki/File:Naxos_Taverna.jpg
image_url = "https://upload.wikimedia.org/wikipedia/commons/6/60/Naxos_Taverna.jpg"  #@param
downloaded_image_path = download_and_resize_image(image_url, 1280, 856, True)

### Desenhar caixas delimitadoras

Para desenvolver o que você viu no laboratório anterior, agora você pode visualizar as caixas delimitadoras previstas, sobrepostas à imagem.  
- Você pode usar `draw_boxes` para fazer isso.  Ele usará `draw_bounding_box_on_image` para desenhar as caixas delimitadoras.

In [ ]:
def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
    """
    Adiciona uma caixa delimitadora a uma imagem.
    
    Args:
        image -- o objeto de imagem
        ymin -- coordenada da caixa delimitadora
        xmin -- coordenada da caixa delimitadora
        ymax -- coordenada da caixa delimitadora
        xmax -- coordenada da caixa delimitadora
        color -- Cor para as bordas da caixa delimitadora
        font -- Fonte para o rótulo da classe
        thickness -- Espessura da borda da caixa delimitadora
        display_str_list -- Rótulos de classe para cada objeto detectado
    
    
    Retornos:
        Sem retorno.  A função modifica o argumento `image 
                    que é passado para essa função
    
    """
    draw = ImageDraw.Draw(image)
    im_width, im_height = image.size
    
    # Dimensione as coordenadas da caixa delimitadora de acordo com a altura e a largura da imageme
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
    
    # Definir as quatro bordas da caixa de detecção
    draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

    # Se a altura total das cadeias de caracteres de exibição adicionadas à parte superior da caixa
    # da caixa delimitadora exceder a parte superior da imagem, empilhe as cadeias de caracteres abaixo da caixa delimitadora
    # em vez de acima.
    #display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
    display_str_heights = [font.getbbox(ds)[3] for ds in display_str_list]
    # Cada display_str tem uma margem superior e inferior de 0,05x.
    total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

    if top > total_display_str_height:
        text_bottom = top
    else:
        text_bottom = top + total_display_str_height
        
    # Inverta a lista e imprima de baixo para cima.
    for display_str in display_str_list[::-1]:
        #text_width, text_height = font.getsize(display_str)
        bbox = font.getbbox(display_str)
        text_width, text_height = (bbox[2],bbox[3])
        margin = np.ceil(0.05 * text_height)
        draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                        (left + text_width, text_bottom)],
                       fill=color)
        draw.text((left + margin, text_bottom - text_height - margin),
                  display_str,
                  fill="black",
                  font=font)
        text_bottom -= text_height - 2 * margin


def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
    """
    Sobreponha caixas rotuladas em uma imagem com pontuações formatadas e nomes de rótulos.
    
    Args:
        image -- a imagem como uma matriz numérica
        boxes -- lista de caixas de detecção
        class_names -- lista de classes para cada objeto detectado
        scores -- números que mostram a confiança do modelo na detecção desse objeto
        max_boxes -- máximo de caixas de detecção a serem sobrepostas na imagem (o padrão é 10)
        min_score -- pontuação mínima necessária para exibir uma caixa delimitadora
    
    Retorna:
        image -- a imagem depois que as caixas de detecção e as classes são sobrepostas na imagem original.
    """
    colors = list(ImageColor.colormap.values())

    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
    except IOError:
        print("Fonte não encontrada, usando a fonte padrão.")
        font = ImageFont.load_default()
    
    for i in range(min(boxes.shape[0], max_boxes)):
        
        # Exibir apenas caixas de detecção que tenham a pontuação mínima ou superior
        if scores[i] >= min_score:
            ymin, xmin, ymax, xmax = tuple(boxes[i])
            display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                         int(100 * scores[i]))
            color = colors[hash(class_names[i]) % len(colors)]
            image_pil = Image.fromarray(np.uint8(image)).convert("RGB")

            # Desenhe uma caixa delimitadora e sobreponha os rótulos de classe na imagem
            draw_bounding_box_on_image(image_pil,
                                       ymin,
                                       xmin,
                                       ymax,
                                       xmax,
                                       color,
                                       font,
                                       display_str_list=[display_str])
            np.copyto(image, np.array(image_pil))
        
    return image

### run_detector

Essa função receberá o modelo de detecção de objetos `detector` e o caminho para uma imagem de amostra e, em seguida, usará esse modelo para detectar objetos.
- Desta vez, run_dtector também chama `draw_boxes` para desenhar as caixas delimitadoras previstas.

In [ ]:
def load_img(path):
    '''
    Carrega uma imagem JPEG e a converte em um tensor.
    
    Args:
        path (string) -- caminho para uma imagem JPEG salva localmente
    
    Retorna:
        (tensor) -- um tensor de imagem
    '''
    
    # lê o arquivo
    img = tf.io.read_file(path)
    
    # converter em um tensor
    img = tf.image.decode_jpeg(img, channels=3)
    
    return img


def run_detector(detector, path):
    '''
    Executa a inferência em um arquivo local usando um modelo de detecção de objetos.
    
    Args:
        detector (model) -- um modelo de detecção de objetos carregado do TF Hub
        path (string) -- caminho para uma imagem salva localmente
    '''
    
    # Carrega um tensor de imagem de um arquivo local path
    img = load_img(path)

    # Adicionar uma dimensão de lote na frente do tensor
    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    
    # Executar a inferência usando o modelo
    start_time = time.time()
    result = detector(converted_img)
    end_time = time.time()

    # Salvar os resultados em um dicionário
    result = {key:value.numpy() for key,value in result.items()}

    # print results
    print("Encontrados %d objetos." % len(result["detection_scores"]))
    print("Tempo de inferência: ", end_time-start_time)

    # Desenhar caixas previstas sobre a imagem
    image_with_boxes = draw_boxes(
      img.numpy(), result["detection_boxes"],
      result["detection_class_entities"], result["detection_scores"])

    # exibir a imagem
    display_image(image_with_boxes)

### Execute o detector em sua imagem selecionada!

In [ ]:
run_detector(detector, downloaded_image_path)

### Execute o detector em mais imagens
Execute a inferência em algumas imagens adicionais de sua escolha e verifique quanto tempo a inferência leva.

In [ ]:
image_urls = [
  # Fonte: https://commons.wikimedia.org/wiki/File:The_Coleoptera_of_the_British_islands_(Plate_125)_(8592917784).jpg
  "https://upload.wikimedia.org/wikipedia/commons/1/1b/The_Coleoptera_of_the_British_islands_%28Plate_125%29_%288592917784%29.jpg",
  # Por Américo Toledano, Source: https://commons.wikimedia.org/wiki/File:Biblioteca_Maim%C3%B3nides,_Campus_Universitario_de_Rabanales_007.jpg
  "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg/1024px-Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg",
  # Fonte: https://commons.wikimedia.org/wiki/File:The_smaller_British_birds_(8053836633).jpg
  "https://upload.wikimedia.org/wikipedia/commons/0/09/The_smaller_British_birds_%288053836633%29.jpg",
  ]

def detect_img(image_url):
    start_time = time.time()
    image_path = download_and_resize_image(image_url, 640, 480)
    run_detector(detector, image_path)
    end_time = time.time()
    print("Inference time:",end_time-start_time)

In [ ]:
detect_img(image_urls[0])

In [ ]:
detect_img(image_urls[1])

In [ ]:
detect_img(image_urls[2])